In [7]:
import datetime
import numpy as np
import networkx as nx
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from itertools import chain
import math
import twittergraph as tg

In [ ]:
# This doesn't do anything yet
# Kindly ignore it

In [11]:
graph = tg.LoadTwitterGraph('/Volumes/pond/Temp/twitter/', 0, n_tweets=10)

Loaded 10 tweets
RTs: 5 


In [9]:
print np.min(tg.timeList)
print np.max(tg.timeList)
split = datetime.datetime(2014, 5, 15) #train/test split

2014-03-15 04:26:31
2014-06-13 22:05:22


In [180]:
jacDict = {}
adamDict = {}
nbrDict = {}
attDict = {}
spDict = {}
for node in G_train.nodes():
    jacDict[node] = {}
    adamDict[node] = {}
    nbrDict[node] = {}
    attDict[node] = {}
    spDict[node] = {}
    
def get_sp(u, v):
    (u,v) = sorted((u,v))
    ed = None
    if G_train.has_edge(u,v):
        ed = G_train.edge[u][v]
        G_train.remove_edge(u,v)
    if v not in spDict[u]:
        try:
            spDict[u][v] = nx.shortest_path_length(G_train, u, v)
        except:
            spDict[u][v] = 1000000
    
    if ed:
        G_train.add_edge(u, v, ed)
    return spDict[u][v]
            
    
def get_jac(u, v):
    (u,v) = sorted((u,v))
    if v not in jacDict[u]:
        j = nx.jaccard_coefficient(G_train, [(u, v)])
        for x,y,p in j:
            jacDict[u][v] = p
    return jacDict[u][v]


def get_adam(u, v):
    (u,v) = sorted((u,v))
    if v not in adamDict[u]:
        j = nx.adamic_adar_index(G_train, [(u, v)])
        try:
            for x,y,p in j:
                adamDict[u][v] = p
        except:
            adamDict[u][v] = 0
    return adamDict[u][v]


def get_adam2(u,v):
    (u,v) = sorted((u,v))
    score = 0
    if v not in adamDict[u]:
        for nbr in nx.common_neighbors(G_train, u, v):
            deg = nx.degree(G_train, nbr)
            if deg >= 2:
                score += 1/math.log(deg)
        adamDict[u][v] = score
    return adamDict[u][v]


def get_att(u,v):
    (u,v) = sorted((u,v))
    if v not in attDict[u]:
        j = nx.preferential_attachment(G_train, [(u, v)])
        for x,y,p in j:
            attDict[u][v] = p
    return attDict[u][v]


def get_nbrs(u, v):
    (u,v) = sorted((u,v))
    if v not in nbrDict[u]:
        nbrs = 0
        for nbr in nx.common_neighbors(G_train, u, v): nbrs += 1
        nbrDict[u][v] = nbrs
    return nbrDict[u][v]
      
        
def all_pairs(graph):
    return chain(graph.edges(), nx.non_edges(graph))

In [181]:
for u,v in all_pairs(G_train):
    print get_sp(u,v)

In [182]:
progress = 0

for n1, n2 in all_pairs(G_train):
    progress += 1
    if progress % 1000000 == 0: print progress
    get_nbrs(n1, n2)
    get_jac(n1, n2)
    get_adam2(n1, n2)
    get_att(n1, n2)
    get_sp(n1, n2)

1000000
2000000


In [188]:
u = []
v = []
has_links = []
jac_co = []
adam = []
att = []
nbrs = []
spl = []
count = 0
for n1, n2 in all_pairs(G_train):
    count += 1
    u.append(n1)
    v.append(n2)
    has_links.append(G_train.has_edge(n1,n2))
    jac_co.append(get_jac(n1,n2))
    adam.append(get_adam2(n1, n2))
    att.append(get_att(n1, n2))
    nbrs.append(get_nbrs(n1, n2))
    spl.append(get_sp(n1, n2))
                
df_train = pd.DataFrame()
df_train['u'] = u
df_train['v'] = v
df_train['link'] = has_links
df_train['jac'] = jac_co
df_train['adam'] = adam
df_train['nbrs'] = nbrs
df_train['att'] = att
df_train['spl'] = spl
print("%d pairs in training set, %d edges" % (count, np.count_nonzero(has_links)))

2800626 pairs in training set, 29830 edges


In [189]:
u = []
v = []
has_links = []
jac_co = []
adam = []
att = []
nbrs = []
spl = []
count = 0

for n1, n2 in nx.non_edges(G_train):
    u.append(n1)
    v.append(n2)
    has_links.append(G.has_edge(n1,n2))
    jac_co.append(get_jac(n1,n2))
    adam.append(get_adam2(n1, n2))
    att.append(get_att(n1, n2))
    nbrs.append(get_nbrs(n1, n2))
    spl.append(get_sp(n1, n2))
    count += 1

df_test = pd.DataFrame()
df_test['u'] = u
df_test['v'] = v
df_test['link'] = has_links
df_test['jac'] = jac_co
df_test['adam'] = adam
df_test['att'] = att
df_test['nbrs'] = nbrs
df_test['spl'] = spl
print("%d pairs in test set, %d true edges" % (count, np.count_nonzero(has_links)))

2770796 pairs in test set, 11205 true edges


In [ ]:
rf = RandomForestClassifier(n_estimators=1000, max_depth=None,
    min_samples_split=2, random_state=0, )
#rf = SVC(C=.00001, gamma=100, probability=True)
fields = ['att']
x_train = df_train.loc[:, fields]
y_train = np.reshape(df_train.link, (-1, 1))

x_test = df_test.loc[:, fields]
classifier = rf.fit(x_train, y_train)
pred = classifier.predict_proba(x_test)

/Users/tomfw/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


In [ ]:
roc_auc_score(np.reshape(df_test.link, (-1, 1)), pred[:,1])